In [1]:
%pip install -q requests beautifulsoup4 pandas

import os
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, unquote


In [2]:
QUERY = "겨울 코디 추천"
MAX_PAGES = 5
OUT_NAME = "naver_blog_links_clean.csv"

UA = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"

print("저장 경로:", os.path.abspath(OUT_NAME))


저장 경로: /content/naver_blog_links_clean.csv


In [3]:
def make_naver_blog_search_url(query: str, start: int = 1) -> str:

    q = requests.utils.quote(query)
    return f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={q}&start={start}"

def fetch_html(url: str) -> str:
    headers = {
        "User-Agent": UA,
        "Accept-Language": "ko-KR,ko;q=0.9",
    }
    r = requests.get(url, headers=headers, timeout=20)
    r.raise_for_status()
    return r.text

def extract_real_url(href: str) -> str:

    try:
        u = urlparse(href)
        qs = parse_qs(u.query)
        if "url" in qs and qs["url"]:
            return unquote(qs["url"][0])
    except Exception:
        pass
    return href


In [4]:
def is_noise_title(title: str) -> bool:

    if not title:
        return True

    t = title.strip()


    if t.replace(",", "").isdigit():
        return True


    if len(t) < 4:
        return True

    return False

def get_blog_links_from_naver_search(html: str):
    soup = BeautifulSoup(html, "html.parser")
    results = []
    seen = set()

    for a in soup.select("a[href]"):
        href = a.get("href", "")
        if not href:
            continue

        real = extract_real_url(href)


        if ("blog.naver.com" in real) or ("m.blog.naver.com" in real) or ("post.naver.com" in real):
            title = a.get_text(" ", strip=True)


            if is_noise_title(title):
                continue

            key = (title, real)
            if key in seen:
                continue
            seen.add(key)

            results.append({"title": title, "url": real})

    return results


In [5]:
all_rows = []

for page in range(1, MAX_PAGES + 1):
    start = 1 + (page - 1) * 10  # 1, 11, 21, 31 ...
    url = make_naver_blog_search_url(QUERY, start=start)

    html = fetch_html(url)
    rows = get_blog_links_from_naver_search(html)

    print(f"[페이지 {page}] 수집: {len(rows)}개")
    all_rows.extend(rows)


df = pd.DataFrame(all_rows).drop_duplicates(subset=["url"]).reset_index(drop=True)
print("전체(중복 제거):", len(df))
df.head(10)


[페이지 1] 수집: 86개
[페이지 2] 수집: 83개
[페이지 3] 수집: 84개
[페이지 4] 수집: 79개
[페이지 5] 수집: 78개
전체(중복 제거): 119


,title,url
0,UGG 1910 호주 여자 어그부츠 추천 국내정식런칭 프리미엄 겨울코디,https://blog.naver.com/minju1004b/224109903723
1,남자패딩 추천 탑텐 아우터 탑시티 겨울 옷 코디 잇템,https://blog.naver.com/minju1004b/224119209847
2,디스커버리 켈리 구스다운 롱다운 8020 고윤정 여성 롱패딩 추천 2025 여자 겨...,https://blog.naver.com/hanbitnuri/224084153914
3,남자 롱패딩 브랜드 내셔널지오그래픽 카이만 구스다운 패딩 점퍼 추천 2025 남성 ...,https://blog.naver.com/hanbitnuri/224094851858
4,여자 숏패딩 추천 2025 지오다노 푸파 여성패딩 겨울 아우터 코디,https://blog.naver.com/hanbitnuri/224119022032
5,"겨울 데일리룩 코디 가 심심하다면? 모자, 안경, 머플러 추천 !",https://blog.naver.com/rudelulu/224139671631
6,지오다노 여성 패딩으로 따뜻한 겨울 코디 (가성비도 좋은 겨울 아우터),https://blog.naver.com/rudelulu/224117959503
7,노스페이스 눕시 패딩 남자 겨울 코디 이번 시즌도 활용!,https://blog.naver.com/coolps8/224128263823
8,남자 겨울 니트 코디 브랜드 추천 키뮤어 민트 컬러 은은한 포인트!,https://blog.naver.com/coolps8/224138135442
9,남자 겨울 코디 추천 코트 패딩 내년에도 무조건 입을 아우터 모음,https://blog.naver.com/coolps8/224120673103


In [6]:
out_path = os.path.abspath(OUT_NAME)
df.to_csv(out_path, index=False, encoding="utf-8-sig")

print("저장 완료:", out_path)
print("파일 존재:", os.path.exists(out_path))


저장 완료: /content/naver_blog_links_clean.csv
파일 존재: True
